# Preprocessing the Data
---

The first model I plan to make will utilize temperature data from the surface mooring and CTD-O variables from the 200 meter platform in 2017. The steps that need to be completed before modeling include: 
* find proportion of missing data - then drop or impute it accordingly
* resample to lower resolution to reduce number of observations to pass to model
* create new dataframe containing the target variable and features
* save clean dataframe as csv

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns

from scipy import stats

sns.set()

---
### Load data

Read in the surface mooring and 200m platform data.

In [ ]:
METBK_data = pd.read_csv('../../coastal_upwelling_output/metbk_data_2017.csv')
platform_data = pd.read_csv('../../coastal_upwelling_output/platform_data_2017.csv')

In [ ]:
CUTI_data = pd.read_csv('../../coastal_upwelling_output/CUTI_daily.csv',
                  parse_dates=[[0,1,2]],
                  infer_datetime_format=True)
CUTI_data.rename(columns={'year_month_day':'time'},inplace=True)

In [ ]:
METBK_data['time'] = pd.to_datetime(METBK_data['time'])
platform_data['time'] = pd.to_datetime(platform_data['time'])

---
### Drop unwanted columns

The goal of this project is to identify upwelling using environmental variables in the ocean, like seawater temperature and salinity. Since this is the goal, we actually don't need the wind data collected by the METBK package. Unforunately, I also have to drop the dissolved oxygen measurements collected by the 200 meter platform due to the instrument malfunctioning or being uncalibrated for a significant period of time in 2017.

I'll be combining the datasets along the column labeled `'time'`, but I'll want to drop that column when I'm done, too.

In [ ]:
METBK_data.drop(columns=['met_windavg_mag_corr_east', 'met_windavg_mag_corr_north'], inplace=True)
METBK_data

In [ ]:
platform_data.drop(columns=['seawater_pressure', 'dissolved_oxygen'], inplace=True)
platform_data

---
### Combine the data

Let's merge these three dataframes into one on the `time` column. Then we won't waste any time looking for nulls or filled values in data that we aren't going to be using. To start, we should slice the METBK dataframe to cover the same time period as the platform data.

In [ ]:
platform_data['time'].tail()

Then use it in a mask to slice the METBK data to match.

In [ ]:
mask = (METBK_data['time'][:] > '2017-01-01') & (METBK_data['time'] <= '2017-09-16')
METBK_data = METBK_data.loc[mask]

mask = (platform_data['time'][:] > '2017-01-01') & (platform_data['time'] <= '2017-09-16')
platform_data = platform_data.loc[mask]

In [ ]:
METBK_data['time'].tail()

Next, we should resample the platform data into 1 hour intervals so it's closer to the time interval seen in the METBK data.

In [ ]:
METBK_data['time'][:20]

In [ ]:
platform_data['time'][:20]

Unfortunately, it looks like the METBK data is collected at inconsistent intervals. I want to see if resampling the 

In [ ]:
platform_min = platform_data.resample('T', on='time').mean().dropna(how='all').reset_index()
print(platform_min.shape)

In [ ]:
METBK_min = METBK_data.resample('T', on='time').mean().dropna(how='all').reset_index()
print(METBK_min.shape)

In [ ]:
platform_min['time']

In [ ]:
METBK_min['time']

In [ ]:
indices = []
for i in range(len(METBK_min['time'])):
    if METBK_min['time'][i] != platform_min['time'][i]:
        indices.append(i)

In [ ]:
len(indices)

In [ ]:
platform_min

In [ ]:
METBK_min

In [ ]:
df = pd.merge(platform_min, METBK_min, on='time', how='right')

In [ ]:
print(platform_min.shape)
print(METBK_min.shape)
print(df.shape)

Let's check out the percentage of our data that is filled with nulls:

In [ ]:
100 * df.isna().sum() / len(df) 

In [ ]:
df['seawater_temperature']

In [ ]:
df

In [ ]:
df.loc[df['seawater_temperature'] < 0]

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna().reset_index(drop=True)

In [ ]:
df

In [ ]:
fig, (ax2, ax1) = plt.subplots(2,1, sharex=True, figsize = (24, 12)) 
ax1.scatter(x=df['time'], y=df['seawater_temperature'], c='g', s=1)
ax1.set_ylabel('Seawater Temperature at 200 meters', size=24)
ax1.set_title('Platform data', size=24)

ax2.scatter(x=df['time'], y=df['sea_surface_temperature'], c='b', s=1)
ax2.set_xlabel('Time', size=24)
ax2.set_ylabel('Sea Surface Temperature', size=24)
ax2.set_title('METBK data', size=24)


plt.xticks(rotation=35);
plt.tight_layout()

In the plot above we can see some areas where data is missing, but this is less than 5% of our total data so hopefully we'll survive! Next, we need to append the CUTI index data to this dataframe.

---
### Appending the CUTI index

Now we can add the upwelling index to this dataframe. First, we only need the upwelling index for the year 2017 at the location 44 degrees North, so let's grab that slice.

In [ ]:
CUTI_data = CUTI_data[CUTI_data['time'].dt.year == 2017][['time', '44N']].reset_index(drop=True)

In [ ]:
CUTI_data

Now there are a few different ways to append this data to our dataframe. The time column in the merged dataframe is on a 1-minute resolution, but the upwelling index is on a 1-day resolution. To combine these, I loop through every month and day in our dataframe (January 01 to September 15) and grab the indices for all measurements taken on that particular day. Then I find the CUTI index value for that same date, and append it to a CUTI column in the dataframe for each date. Looping through the months and days turned out to be hugely more efficient than looping through each row in the dataframe one by one, I'm very happy with this solution!

In [ ]:
for month in range(1, 10): # our dataframe only contains dates up to 2017-09-15
    for day in range(1, 32):
        try:
            indices = df[(df['time'].dt.month == month) & (df['time'].dt.day == day)].index
            cuti_value = CUTI_data.loc[(CUTI_data['time'].dt.month == month) & (CUTI_data['time'].dt.day == day)]['44N'].values[0]
            #print(month, day, cuti_value)
            df.loc[indices,'CUTI'] = cuti_value
        except:
            pass

In [ ]:
df

There, now we've got the CUTI upwelling index for each measurement taken. We can one hot encode this to a binary value, because we'll be building a classifier model with two classes: upwelling, or not upwelling. To binarize this, we can say that any row where the upwelling index is positive is a 1, and every other row is a 0.

In [ ]:
df['upwelling'] = df['CUTI'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
df['upwelling'].value_counts(normalize=True)

It looks like we have favorable conditions for upwelling for about 61.28% in 2017 from January 1st to September 15th.

In [ ]:
df

Here's the final dataframe! Now that everything has been properly appended along time, we can drop this column.

In [ ]:
df.drop(columns=['time'], inplace=True)
df

---
### Removing outliers

Statistical analysis is a vital piece of the preprocessing process! There are a few different ways to remove outliers and you can read about them [in this article](https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba). I'm going to use the z-score to identify and remove outliers in this data.

Question: should I be removing outliers before grouping the data into upwelling/not upwelling? Aren't I artifically labeling the data here and impacting what might be considered outliers? Answer to this problem: run the z-score on the entire dataset, not on subsets grouped by upwelling/not upwelling. That way my labels aren't impacting which datapoints are identified as outliers.

These datasets are too large to put into a scatter plot, but some box plots will give us a good idea of our data distributions. 

In [ ]:
meanprops={"marker":"o",
                       "markerfacecolor":"white", 
                       "markeredgecolor":"black",
                      "markersize":"10"}

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24,12))
sns.boxplot(y='seawater_temperature', x='upwelling', data=df, ax=ax1, showmeans=True, meanprops=meanprops)
sns.boxplot(y='practical_salinity', x='upwelling', data=df, ax=ax2, showmeans=True, meanprops=meanprops);

First of all, look at these wonderful signals of upwelling in our data! Upwelled water is colder and saltier than the alternative, just like we'd expect! But there some potential outliers here, and some pretty significant looking ones in the salinity data in particular. We should check sea surface temperatures and salinity as well. 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24,12))
sns.boxplot(y='density', x='upwelling', data=df, ax=ax1, showmeans=True, meanprops=meanprops)
sns.boxplot(y='sea_surface_temperature', x='upwelling', data=df, ax=ax2, showmeans=True, meanprops=meanprops);

Interesting that the sea surface temperature has an inverse relationship to the seawater temperature at depth. This might indicate that upwelling doesn't significantly impact the surface waters, and it's effect is limited to lower in the water column. It'll be especially interesting to see how this changes in the profiler data at various depths, but that will be for another time. For now, let's get the z-scores so we can identify and remove outliers.

In [ ]:
X = df.drop(columns=['CUTI', 'upwelling'])
X

In [ ]:
z = np.abs(stats.zscore(X))
print(z)

In [ ]:
threshold = 3
print(np.where(z > 3))

In [ ]:
print(z[29655][1])

In [ ]:
clean_df = df[(z < 3).all(axis=1)]

In [ ]:
print(df.shape)
print(clean_df.shape)

In [ ]:
pct_outliers = 100 * (df.shape[0] - clean_df.shape[0]) / df.shape[0] 
print(f'Outliers made up {round(pct_outliers, 4)}% of the data.')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24,12))
sns.boxplot(y='seawater_temperature', x='upwelling', data=clean_df, ax=ax1, showmeans=True, meanprops=meanprops)
sns.boxplot(y='practical_salinity', x='upwelling', data=clean_df, ax=ax2, showmeans=True, meanprops=meanprops);

First of all, look at these wonderful signals of upwelling in our data! Upwelled water is colder and saltier than the alternative, just like we'd expect! But there some potential outliers here, and some pretty significant looking ones in the salinity data in particular. We should check sea surface temperatures and salinity as well. 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24,12))

sns.boxplot(y='density', x='upwelling', data=clean_df, ax=ax1, showmeans=True, meanprops=meanprops)
sns.boxplot(y='sea_surface_temperature', x='upwelling', data=clean_df, ax=ax2, showmeans=True, meanprops=meanprops);

Hey, that looks much better! Obviously there are still some data points considered outliers by the standard boxplot, but I'm satisfied with the z-score method. 

---
### Saving the clean data

This dataset might be ready for modeling now! Is there anything I'm still missing? 

In [ ]:
clean_df.to_csv('../../coastal_upwelling_output/clean_dataframe.csv', index=False)